<a href="https://colab.research.google.com/github/gabilodeau/INF6804/blob/master/siamfc_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INF6804 Vision par ordinateur

Polytechnique Montréal

Author: Hughes Perreault

siamfc visual object tracker example.

Original GitHub: https://github.com/huanglianghua/siamfc-pytorch

Install the required packages.


In [1]:
pip install torch torchvision got10k opencv-python

Clone the repo.

In [2]:
import os
# get siamfc-pytorch source
if not os.path.exists('siamfc-pytorch'):
  !git clone https://github.com/huanglianghua/siamfc-pytorch.git

Download the pretrained weights.

In [3]:
pretrained_file_name = 'siamfc_alexnet_e50.pth'
if not os.path.exists(pretrained_file_name):
  !gdown https://drive.google.com/uc?id=1UdxuBQ1qtisoWYFZxLgMFJ9mJtGVw6n4

Download and unzip the DragonBaby sequence from the Visual Tracker Benchmark. 

In [4]:
file_name = 'DragonBaby.zip'
if not os.path.exists(file_name):
  !gdown http://cvlab.hanyang.ac.kr/tracker_benchmark/seq/DragonBaby.zip
if not os.path.exists('DragonBaby'):
  !unzip DragonBaby.zip

Update the path.


In [5]:
import sys
sys.path.append('siamfc-pytorch/')

Import the necessary libraries.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
import cv2
import os
from collections import namedtuple
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from got10k.trackers import Tracker
from siamfc import ops
from siamfc.backbones import AlexNetV1
from siamfc.heads import SiamFC
from siamfc.losses import BalancedLoss
from siamfc.datasets import Pair
from siamfc.transforms import SiamFCTransforms
from siamfc.siamfc import Net
from siamfc.siamfc import TrackerSiamFC
import glob

Define the paths for the images, the starting rectangle and the pretrained weights. Then execute the tracker on the images. 

In [7]:
seq_dir = os.path.expanduser('DragonBaby/')
img_files = sorted(glob.glob(seq_dir + 'img/*.jpg'))
anno = np.loadtxt(seq_dir + 'groundtruth_rect.txt', delimiter=',', usecols=range(4))

net_path = 'siamfc_alexnet_e50.pth'
tracker = TrackerSiamFC(net_path=net_path)
boxes, times = tracker.track(img_files, anno[0], visualize=False)

For each box, print it on the corresponding image. Stop after NBR_FRAMES. 

In [8]:
from google.colab.patches import cv2_imshow

NBR_FRAMES = 25
for index, box in enumerate(boxes):
  if index > NBR_FRAMES:
    break
  image_path = os.path.join(seq_dir, 'img/' + str(index+1).zfill(4) + '.jpg')
  image = cv2.imread(image_path)
  box = [int(item) for item in box]
  cv2.rectangle(image, (box[0], box[1]), (box[0]+box[2], box[1]+box[3]), (255, 0, 0), 2) 
  cv2_imshow(image)


Output hidden; open in https://colab.research.google.com to view.